In [18]:
import pickle
import sys
import matplotlib
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
import matplotlib.ticker as mtick
import numpy as np
import pandas as pd
import datetime
import seaborn as sns
from tabulate import tabulate


sys.path.append("../")
sys.path.append("../covid19_inference")

import covid19_soccer
from covid19_soccer.plot.utils import get_from_trace
import covid19_inference as cov19

In [6]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [7]:
""" Matplotlib config
"""
matplotlib.rcParams.update(matplotlib.rcParamsDefault)
matplotlib.rcParams['font.family'] = "sans-serif"
matplotlib.rcParams["figure.figsize"] = [3.4, 2.7]  # APS single column
matplotlib.rcParams["figure.dpi"] = 300  # this primarily affects the size on screen
#matplotlib.rcParams['axes.linewidth'] = 0.3
matplotlib.rcParams["axes.labelcolor"] = "black"
matplotlib.rcParams["axes.edgecolor"] = "black"
matplotlib.rcParams["xtick.color"] = "black"
matplotlib.rcParams["ytick.color"] = "black"
matplotlib.rcParams["xtick.labelsize"] = 8
matplotlib.rcParams["ytick.labelsize"] = 8
matplotlib.rcParams["axes.labelsize"] = 8
matplotlib.rcParams["axes.titlesize"]= 10
matplotlib.rcParams["legend.fontsize"] = 6
matplotlib.rcParams["legend.title_fontsize"] = 8

""" Colors
Done with rcParams / see plot.rcParams
"""


""" General configs
"""
fig_path = "./figures"
countries = ['England', 
         "Czechia",
         "Italy",
         'Scotland',
         "Spain",
         "Germany",
         "France",
         "Austria", 
         'Belgium', 
         "Slovakia", 
         "Portugal", 
         "Netherlands"]
xlim_ts = [datetime.datetime(2021,5,30),datetime.datetime(2021,7,23)] # Plotting range
tune = 500
draws = 1000 
max_treedepth = 12
offset = 0 #offset of soccer games, mainly for sanity checks
width_prior_delay = 0.1

In [15]:
def load(fstr):
    with open(fstr, "rb") as f:
         return pickle.load(f)

models,traces,dls = {},{},{}
for country in countries:
    # Load trace, model and dataloader
    #'UEFA-beta=False-country=France-offset_games=0-draw_delay=True-weighted_alpha_prior=False-prior_delay=-1-tune=1000-draws=1500-max_treedepth=12.pickled'
    #'UEFA-beta=False-country=France-offset_games=0-draw_delay=True-weighted_alpha_prior=0-prior_delay=-1-width_delay_prior=0.1-sigma_incubation=-1.0-median_width_delay=1.5-tune=200-draws=300-max_treedepth=10.pickled'

    fstr=(f"/data.nst/jdehning/covid_uefa_traces11/UEFA"+
        f"-beta=False"+
        f"-country={country}"+
        f"-offset_data={offset}"+
        f"-draw_delay=True"+
        f"-weighted_alpha_prior=0"+
        f"-prior_delay={-1}"+
        f"-width_delay_prior={width_prior_delay}"+
        f"-sigma_incubation=-1.0"+
        f"-median_width_delay=1.0"+
        f"-interval_cps=10.0"+
        f"-tune={tune}"+
        f"-draws={draws}"+
        f"-max_treedepth={max_treedepth}.pickled")
    model, trace = load(fstr)
    dl = covid19_soccer.dataloader.Dataloader_gender(countries=[country])
    models[country]=model
    traces[country]=trace
    dls[country]=dl

In [24]:
begin = datetime.datetime(2021, 6, 11)
end = datetime.datetime(2021, 7, 11)

table = []
for country in countries:
    infections_base, infections_alpha = covid19_soccer.plot.other.get_alpha_infections(traces[country], models[country], dls[country]) 
    i_begin = (begin - models[country].sim_begin).days
    i_end = (end - models[country].sim_begin).days + 1  # inclusiv last day

    # Sum over the choosen range (i.e. month of uefa championship)
    num_infections_base = np.sum(infections_base[..., i_begin:i_end, :], axis=(-2,-1))
    num_infections_alpha = np.sum(infections_alpha[..., i_begin:i_end, :], axis=(-2,-1))

    # Create pandas dataframe for easy violin plot
    percentage_soccer = num_infections_alpha / (
        num_infections_base + num_infections_alpha
    )*100
    CI = np.percentile(percentage_soccer, q = (2.5,50,97.5))
    num_samples = np.prod(percentage_soccer.shape)
    prob_positive = np.sum(percentage_soccer > 0)/num_samples*100
    if prob_positive > 99.9:
        text_prob_pos = "$> 99.9$\%"

    else:
        text_prob_pos = f"{prob_positive:.1f}\%"
    
    table.append([country, f"{CI[1]:.1f}\% (CI: [{CI[0]:.1f}\%, {CI[2]:.1f}\%])", text_prob_pos])


headers = ['Country', 'median fraction soccer related cases', 'probability soccer increased cases']
print(tabulate(table, headers, tablefmt="latex_raw"))

\begin{tabular}{lll}
\hline
 Country     & median fraction soccer related cases   & probability soccer increased cases   \\
\hline
 England     & 8.9\% (CI: [6.1\%, 11.6\%])            & $> 99.9$\%                           \\
 Czechia     & 5.5\% (CI: [0.7\%, 11.4\%])            & 98.8\%                               \\
 Italy       & 6.3\% (CI: [-4.6\%, 15.8\%])           & 87.7\%                               \\
 Scotland    & 4.3\% (CI: [3.2\%, 5.6\%])             & $> 99.9$\%                           \\
 Spain       & 1.8\% (CI: [-0.5\%, 3.8\%])            & 95.5\%                               \\
 Germany     & 1.1\% (CI: [-1.9\%, 3.7\%])            & 79.3\%                               \\
 France      & 0.8\% (CI: [-0.5\%, 2.5\%])            & 88.8\%                               \\
 Austria     & 0.6\% (CI: [-3.0\%, 4.3\%])            & 63.8\%                               \\
 Belgium     & 0.4\% (CI: [-2.0\%, 2.7\%])            & 63.8\%                               \\
 Slov